import pandas 

In [205]:
import pandas as pd
import sentence_encoder

In [6]:
bing_data=pd.read_csv("../data/bing.csv",header=None)

In [132]:
import numpy as np
getty_data_pd=pd.read_csv("../data/cleaned_getty_data.csv",index_col=0)

getty_data_pd = getty_data_pd.replace(np.nan, "", regex=True)

In [204]:
getty_data_pd=getty_data_pd.reset_index()

In [ ]:
big_encoder=sentence_encoder.SentenceTransform("roberta-base-nli-stsb-mean-tokens")

In [187]:
glove_encoder=sentence_encoder.SentenceTransform("average_word_embeddings_glove.6B.300d")

In [94]:
mean_caption_embedding=mega.view(6650,768).mean(axis=1)
mean_caption_embedding_numpy=[mean_caption_embedding.detach().numpy()]



In [165]:
#mean for full sentences 

def embed_mean(word_embedding_model,text_column:list,max_number:int):
    counter=0
    all_captions=[]
    for caption in text_column:
        if counter>max_number:
            break 
        if len(caption)==0:
            continue
        else:
            result=big_encoder.extract_text_features_simple([caption])
            all_captions.append(result)
        counter+=1

    mega=torch.Tensor(all_captions)
    return mega.view(mega.shape[0],mega.shape[2]).mean(dim=0)


def embed_column(word_embedding_model, text_column:list, mean_embedd, max_number:int):
    
    counter=0
    all_captions=[]
    for caption in getty_data_pd["caption"].values:
        if counter>max_number:
            break 
        if len(caption)==0:
            all_captions.append([mean_embedd])

        else:
            result=big_encoder.extract_text_features_simple([caption])
            all_captions.append(result)
        counter+=1
    return all_captions

In [ ]:
mean_embedding=embed_mean(big_encoder,getty_data_pd["caption"].values,50)
all_data=embed_column(big_encoder,getty_data_pd["caption"].values,mean_embedding,15)

In [211]:
getty_data_pd["best_tags"][0]

'Opioid   Syringe   Addiction   Despair   Heroin   Addict   Drug Overdose   Recreational Drug   Teenager   IV Drip   Abuse   Adult Amphetamine   Close-Up   Cocaine   Criminal   Danger   Dependency   Depression - Sadness   Emotional Stress   Forbidden   Frustration   Grief   Hand   Healthcare And Medicine   Horizontal   Human Body Part   Human Hand   Illness   Photography   Problems   Risk   Sharp   Snorting   Thailand   Vein   Worried   Photos'

In [222]:
result=glove_encoder.extract_text_features_simple(getty_data_pd["best_tags"][0].replace("  "," ").split(" "))

In [274]:

def embed_glove_tags(glove_embedding_model,tag_column_list,mean_embed, max_number):
    final_data=[]
    for i in range(max_number):
        tags_to_look_at=tag_column_list[i]
       
        
        result=_select_tags_glove(glove_embedding_model,tags_to_look_at)
  
        result=torch.Tensor(result).mean(dim=0).detach().numpy()
        final_data.append(result)
        
    return final_data
        
def _select_tags_glove(glove_embedding_model,tag_string:str):
    #just take out the tags that are actual words 
    tags=tag_string.replace("  "," ").split(" ")

    sub_tags=[]
    for tag in tags: 
        if(tag!=""):
            sub_tags.append(tag)
    
    
    temp_result=glove_embedding_model.extract_text_features_simple(sub_tags)
    final_final=[]

    for temp in temp_result:
        if temp[0]!=0:
  
            final_final.append(temp)
 
    return final_final



In [277]:
result=embed_glove_tags(glove_encoder,getty_data_pd["best_tags"].values,None,1000)

torch.Size([1000, 300])

In [3]:
cls1=["Ethnicity","Religion","Sexual Orientation","Gender","Gender Identity","Disability Disease","Nationality",
      "Immigration Status"]
ex1=["African","Muslim","Lesbian","Male","Transgender","Autistic","Nationality","Migrant","Poor"]
ex2=["White","Christian","Gay","Woman","Queer","Cancer","Nationality","Undocumented","Rich"]
    
    

In [4]:
class ProtectedClassifierSimple():
    
    def __init__():
        cls1=["Ethnicity","Religion","Sexual Orientation","Gender","Gender Identity","Disability Disease","Nationality",
      "Immigration Status"]
        ex1=["African","Muslim","Lesbian","Male","Transgender","Autistic","Nationality","Migrant","Poor"]
        ex2=["White","Christian","Gay","Woman","Queer","Cancer","Nationality","Undocumented","Rich"]
        
        self.class_names=cls1
        
        self.comparison_names=[]
        self.comparison_names.append(ex1)
        self.comparison_names.append(ex2)
        